**ASCII文件处理**

@Author: Ray  
@Time: 2023-09-04  
@Cite: https://docs.astropy.org/en/stable/io/ascii/index.html

In [8]:
from pathlib import Path
import time

path_test = Path("/Users/rui/Code/1_Astronote/02_Astropy/example/3_3_ascii")

---
## 读取表格
* [ascii.read()](https://docs.astropy.org/en/stable/api/astropy.io.ascii.read.html#astropy.io.ascii.read)
* 全部功能可由更高阶的封装[Table()](https://docs.astropy.org/en/stable/table/index.html#astropy-table)实现

In [9]:
from astropy.io import ascii
data = ascii.read(path_test / "sources.dat")
print(type(data))
data

<class 'astropy.table.table.Table'>


obsid,redshift,X,Y,object
int64,float64,int64,int64,str11
3102,0.32,4167,4085,Q1250+568-A
877,0.22,4378,3892,Source 82


In [10]:
# 使用converters参数，将数据转换为合适的类型
ascii.read(path_test / "sources.dat", converters={"Y": str})

obsid,redshift,X,Y,object
int64,float64,int64,str4,str11
3102,0.32,4167,4085,Q1250+568-A
877,0.22,4378,3892,Source 82


### 读取SExtractor产生的数据表

In [11]:
path = path_test / "sextractor.cat"

# 方法1
t = time.time()
from astropy.table import Table
tbl1 = Table.read(path, format="ascii.sextractor")
print(time.time() - t)

# 方法2
t = time.time()
from astropy.io import ascii
tbl2 = ascii.read(path, format="sextractor")
print(time.time() - t)

3.7927610874176025
3.8666610717773438


---
## 写入表格
* [ascii.write()](https://docs.astropy.org/en/stable/api/astropy.io.ascii.write.html#astropy.io.ascii.write)
* 全部功能可由更高阶的封装[Table()](https://docs.astropy.org/en/stable/table/index.html#astropy-table)实现
* 表格数据推荐使用[ECSV格式](https://docs.astropy.org/en/stable/io/ascii/ecsv.html#ecsv-format)，这种格式相比于传统csv，可以存储更多的元数据

In [12]:
import numpy as np
from astropy.table import Table

# 生成数据
data = Table()
data['x'] = np.array([1, 2, 3,], dtype=np.int32)
data['y'] = data['x'] ** 2
data

x,y
int32,int32
1,1
2,4
3,9


In [13]:
# 使用ascii.write()函数将数据写入文件
ascii.write(data, path_test / "write_func_test.dat", overwrite=True)

# 使用ECSV格式存储表格
data.write(path_test / "write_func_test.ecsv", overwrite=True)

### 机器可读表（MRT）
* Machine-Readable Table (MRT)

In [14]:
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn
from astropy import units as u

table = Table()
table['Name'] = ['ASASSN-15lh', 'ASASSN-14li']

# MRT Standard requires all quantities in SI units.
temperature = [0.0334, 0.297] * u.K
table['Temperature'] = temperature.to(u.keV, equivalencies=u.temperature_energy())
table['nH'] = Column([0.025, 0.0188], unit=u.Unit(10**22))
table['Flux'] = ([2.044 * 10**-11] * u.erg * u.cm**-2).to(u.Jy * u.Unit(10**12))
table['Flux'] = MaskedColumn(table['Flux'], mask=[True, False])
table['magnitude'] = [u.Magnitude(25), u.Magnitude(-9)]

from astropy.time import Time, TimeDelta
from astropy.timeseries import TimeSeries
ts = TimeSeries(time_start=Time('2019-01-01'), time_delta=2*u.day, n_samples=1)
table['Obs'] = Column(ts.time.decimalyear, description='Time of Observation')
table['Cadence'] = Column(TimeDelta(100.0, format='sec').datetime.seconds, unit=u.s)

from astropy.coordinates import SkyCoord
table['coord'] = [SkyCoord.from_name('ASASSN-15lh'),
                  SkyCoord.from_name('ASASSN-14li')]  
table.write(path_test / 'coord_cols.dat', format='ascii.mrt', overwrite=True) 
    
table['coord'] = table['coord'].geocentrictrueecliptic  
table['Temperature'].format = '.5E' # Set default column format.
table.write(path_test / 'ecliptic_cols.dat', format='ascii.mrt', overwrite=True)  

table

Name,Temperature,nH,Flux,magnitude,Obs,Cadence,coord
,keV,1e+22,1e+12 Jy,,,s,"deg,deg,None"
str11,float64,float64,float64,float64,float64,int64,SkyCoord
ASASSN-15lh,2.87819E-09,0.025,--,1e-10,2019.0,100,"306.2242089804817,-45.621592997378514,0.9999999999999999"
ASASSN-14li,2.55935E-08,0.0188,2.044,3981.0717055349733,2019.0,100,"183.75498618886257,21.051406891826403,1.0"


## 固定宽度的表格
* [Fixed-Width Gallery](https://docs.astropy.org/en/stable/io/ascii/fixed_width_gallery.html#fixed-width-gallery)
* 可用于程序中花哨的输出显示

In [15]:
# 生成数据
data = Table()
data['x'] = np.array([1, 2, 3,], dtype=np.int32)
data['y'] = data['x'] ** 200
data

x,y
int32,int32
1,1
2,0
3,-1426542431


In [16]:
ascii.write(data, format='fixed_width_two_line')

x           y
- -----------
1           1
2           0
3 -1426542431


In [17]:
ascii.write(data, format='fixed_width_two_line', bookend=True, delimiter='|')

|x|          y|
|-|-----------|
|1|          1|
|2|          0|
|3|-1426542431|


In [18]:
ascii.write(data, format='fixed_width_two_line',
                 delimiter_pad=' ', position_char='=')

x             y
=   ===========
1             1
2             0
3   -1426542431


## 将表格输出为LaTeX格式
* https://docs.astropy.org/en/stable/api/astropy.io.ascii.Latex.html#latex

In [19]:
from astropy.io import ascii
data = {'name': ['bike', 'car'], 'mass': [75,1200], 'speed': [10, 130]}
ascii.write(data, Writer=ascii.Latex,
                 latexdict = {'units': {'mass': 'kg', 'speed': 'km/h'}})

\begin{table}
\begin{tabular}{ccc}
name & mass & speed \\
 & kg & km/h \\
bike & 75 & 10 \\
car & 1200 & 130 \\
\end{tabular}
\end{table}
